## Alternative Model (Preprocessing)
You need to run this before you run notebook 4b.

The wavs in your dataset will be converted to 9bit linear and 80-band mels.

In [1]:
#from multiprocess import Pool, cpu_count
import math, pickle, os, glob
import numpy as np
from utils.display import *
from utils.dsp import *
import python_speech_features as psf
import scipy.io.wavfile as wavfile
from tqdm import tqdm
import python_speech_features as psf

In [2]:
bits = 9
notebook_name = 'nb4'

In [3]:
# Point SEG_PATH to a folder containing your training wavs 
# Doesn't matter if it's LJspeech, CMU Arctic etc. it should work fine
SEG_PATH = f'en/wavs'
SEG_PITCH_PATH = f'en/pitch'
DATA_PATH = f'data'

In [4]:
def get_files(path, extension='.wav') :
    filenames = []
    for filename in glob.iglob(f'{path}/**/*{extension}', recursive=True):
        filenames += [filename]
    return filenames


In [5]:
wav_files = get_files(SEG_PATH)
len(wav_files)

13100

In [6]:
def load_pitch(path):
    with open(path) as f:
        X = f.read()
    X=X.split("\n")
    X=X[7:len(X)-1]
    array = []
    for line in X:
        array.append(line.split(" ")[2])
    array = np.array(array)
    array = array.astype(np.float)
    return array

def convert_file(path) :
    wav = load_wav(path, encode=False)
    rate,sig = wavfile.read(path)
    mel = psf.mfcc(sig,rate,winlen=win_length/sample_rate,winstep=hop_length/sample_rate,nfft=n_fft).T
    #mel = melspectrogram(wav)
    quant = (wav + 1.) * (2**bits - 1) / 2
    return mel.astype(np.float32), quant.astype(np.int)

def upsampling(pitch, out_size,sample_rate=22050,pitch_interval=0.005):
    out_pitch = []
    for i in range(out_size):
        t = i/sample_rate
        #print(t)
        index = int(t/pitch_interval)
        try:
            out_pitch.append(pitch[index])
        except:
            #print(t)
            out_pitch.append(pitch[-1])
    return np.array(out_pitch)

In [7]:
#wL = [0.005,0.01,0.02,0.03]
wS = [0.00001*i for i in range(10,1000)]
#wav = load_wav(wav_files[1])
print(len(sig))
rate,sig = wavfile.read(wav_files[1])
min_winstep = -1
min_diff = 10
for i,winstep in enumerate(wS):
    mel = psf.mfcc(sig,rate,winlen=0.01,winstep=winstep,nfft=n_fft).T
    diff = abs(round(len(sig)/mel.shape[-1])-len(sig)/mel.shape[-1])
    if(diff<min_diff):
        min_diff=diff
        min_i = winstep

NameError: name 'sig' is not defined

We determine the "magic numbers" to get a (almost) integer ratio between the size of the initial signal, and the size of the MFCC vector.

We have `win_len=0.01`, and `winstep = 0.0053` that leads to the ratio from mel_size to sig size is $1:117$

In [7]:
QUANT_PATH = DATA_PATH + '/quant/'
MEL_PATH = DATA_PATH + '/mel/'
PITCH_PATH = DATA_PATH + '/pitch/'
#%mkdir -p $QUANT_PATH
#%mkdir -p $MEL_PATH

In [8]:
def process_wav(path) :
    id = path.split('/')[-1][:-4].split("\\")[-1]
    m, x = convert_file(path)
    pitch = load_pitch(f'{SEG_PITCH_PATH}/{id}.f0')
    new_pitch = upsampling(pitch, len(x))
    np.save(f'{PITCH_PATH}{id}.npy', new_pitch)
    np.save(f'{MEL_PATH}{id}.npy', m)
    np.save(f'{QUANT_PATH}{id}.npy', x)
    return id

In [9]:
process_wav(wav_files[0])

'LJ001-0001'

In [10]:
# This will take a while depending on size of dataset
#pool = Pool(processes=cpu_count())
#dataset_ids = []
#for i, id in enumerate(pool.imap_unordered(process_wav, wav_files), 1):
#    dataset_ids += [id]
#    stream('Processing: %i/%i', (i, len(wav_files)))

In [11]:
# This will take a while depending on size of dataset
dataset_ids = []
for elt in tqdm(wav_files):
    dataset_ids+=[process_wav(elt)]

100%|██████████████████████████████████████████████████████████████████████████| 13100/13100 [1:38:36<00:00,  2.68it/s]


In [12]:
with open(DATA_PATH + '/dataset_ids.pkl', 'wb') as f:
    pickle.dump(dataset_ids, f)

In [13]:
len(dataset_ids)

13100